
# Capstone: Find the best neighborhood in Toronto to open a Restaurant Supply Store
## 1. Load all the Data from all the various sources.
### 1.1 Toronto neighborhoods broken down by postal code.

In [1]:
# Load the required libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Found the table using beautifulsoup and used Pandas to read it in. 
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))


# WRANGLE/Transform THE DATA
# Convert the list back into a dataframe
data = pd.DataFrame(df[0])

# Rename the columns as instructed
data = data.rename(columns={0:'PostalCode', 1:'Bourough', 2:'Neighborhood'})

# Get rid of the first row which contained the table headers from the webpage
data = data.iloc[1:]


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
data = data[~data['Bourough'].str.contains('Not assigned')]


# More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice 
#that M5A is listed twice and has two neighborhoods: Harbourfront 
#and Regent Park. These two rows will be combined into one row with 
#the neighborhoods separated with a comma
df2=data.groupby(['PostalCode', 'Bourough']).apply(lambda group: ', '.join(group['Neighborhood']))


# Convert the Series back into a DataFrame and put the 'Neighbourhood' column label back in
df2=df2.to_frame().reset_index()
df2 = df2.rename(columns={0:'Neighborhood'})

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df2.loc[df2.Neighborhood == 'Not assigned', 'Neighborhood' ] = df2.Bourough

# Display the DataFrame
df2.head()

,PostalCode,Bourough,Neighborhood
0,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights ...
1,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University)
2,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto)
3,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West
4,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central



### 1.1.1 Load Toronto geospatial cooridinates and merge to Toronto Postal Code Data

In [2]:
# Load Toronto geospatial cooridinates
!wget -O to_geo_space.csv http://cocl.us/Geospatial_data

#Read into dataframe
gf = pd.read_csv('to_geo_space.csv')

#rename the coloumns so the match
gf = gf.rename(columns={'Postal Code':'PostalCode'})

#Merge the Toronto data with geo cooridinate data
gf_new = pd.merge(df2, gf, on='PostalCode', how='inner')

# display the new dataframe
gf_new.head()

--2021-09-19 07:56:56--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 150.239.80.14, 150.239.82.224
Connecting to cocl.us (cocl.us)|150.239.80.14|:80... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://cocl.us/Geospatial_data [following]
--2021-09-19 07:56:56--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|150.239.80.14|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-09-19 07:56:57--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-09-19 07:56:57--  https://ibm.box.com/public/static/9afzr8

,PostalCode,Bourough,Neighborhood,Latitude,Longitude


In [3]:
# Load this data from Stats Canada
df_pop = pd.read_csv('https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Tables/File.cfm?T=1201&SR=1&RPP=9999&PR=0&CMA=0&CSD=0&S=22&O=A&Lang=Eng&OFT=CSV',encoding = 'unicode_escape')
# Rename the columns appropiatley
df_pop = df_pop.rename(columns={'Geographic code':'PostalCode', 'Geographic name':'PostalCod2', 'Province or territory':'Province', 'Incompletely enumerated Indian reserves and Indian settlements, 2016':'Incomplete', 'Population, 2016':'Population_2016', 'Total private dwellings, 2016':'TotalPrivDwellings', 'Private dwellings occupied by usual residents, 2016':'PrivDwellingsOccupied'})
df_pop= df_pop.drop(columns=['PostalCod2', 'Province', 'Incomplete', 'TotalPrivDwellings', 'PrivDwellingsOccupied'])

# Get rid of the first row 
df_pop = df_pop.iloc[1:]
df_pop.head()

,PostalCode,Population_2016
1,A0A,46587.0
2,A0B,19792.0
3,A0C,12587.0
4,A0E,22294.0
5,A0G,35266.0


### 1.2.1 Merge Toronto Neighbourhood populations data with Toronto Postal Code data 


In [4]:
#Merge the Toronto Pop data with geo postalcode data
gf_new
gf_new = pd.merge(df_pop, gf_new, on='PostalCode', how='right')
# sort on population
gf_new = gf_new.sort_values(by=['Population_2016'], ascending=False)

# display the new dataframe
gf_new.head()

,PostalCode,Population_2016,Bourough,Neighborhood,Latitude,Longitude


#### Key Observation here is a list of Toronto Neighborhoods Populations

### 1.3 Toronto neighborhoods average after tax income broken down by postal code